# 05_evaluation
Evaluación final del modelo (LightGBM + threshold tuning).

In [19]:
import joblib
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report
from sklearn import set_config
from sklearn import set_config
set_config(transform_output="pandas")
from IPython.display import display


In [15]:
# Cargar modelo y datos de prueba
pipeline = joblib.load('../models/model_pipeline.pkl')
X_test = pd.read_csv('../data/processed/X_test.csv')
y_test = pd.read_csv('../data/processed/y_test.csv').values.ravel()

In [16]:
# Probabilidades en TEST
y_proba_test = pipeline.predict_proba(X_test)[:, 1]
best_t = 0.795
y_pred_opt = (y_proba_test >= best_t).astype(int)

In [30]:
# Métricas finales
# Métricas finales
acc = accuracy_score(y_test, y_pred_opt)
f1 = f1_score(y_test, y_pred_opt)
auc = roc_auc_score(y_test, y_proba_test)
cm = confusion_matrix(y_test, y_pred_opt)

print("Métricas finales del modelo en el conjunto de prueba:")

resultados = pd.DataFrame({
    "Métrica": ["Accuracy", "F1 Score", "ROC-AUC"],
    "Valor": [round(acc, 4), round(f1, 4), round(auc, 4)]
})

display(resultados)

print("\nMatriz de Confusión:")
print(cm)

print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_opt))

Métricas finales del modelo en el conjunto de prueba:


,Métrica,Valor
0,Accuracy,0.9742
1,F1 Score,0.6759
2,ROC-AUC,0.9603



Matriz de Confusión:
[[44096   340]
 [  859  1250]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99     44436
           1       0.79      0.59      0.68      2109

    accuracy                           0.97     46545
   macro avg       0.88      0.79      0.83     46545
weighted avg       0.97      0.97      0.97     46545




# Interpretación de Resultados del Modelo (LightGBM + Threshold Tuning)

Este documento presenta una explicación técnica y clara de las métricas finales obtenidas por el modelo de clasificación encargado de predecir si una canción será un **HIT** o **NO HIT**, utilizando un modelo **LightGBM optimizado mediante threshold tuning**.

---

## 1. Métricas Globales del Modelo

| Métrica   | Valor  |
|-----------|--------|
| **Accuracy**  | 0.9742 |
| **F1 Score**  | 0.6759 |
| **ROC-AUC**   | 0.9603 |

### ● Accuracy (97.42%)

El modelo acierta en el **97% de las predicciones**.  
Sin embargo, debido al **fuerte desbalance** entre clases (muchos NO HIT y pocos HIT), esta métrica no es suficiente para evaluar la calidad del modelo.

---

### ● F1 Score (67.59%)

Es la métrica **más importante** para este problema porque combina:

- **Precision**: calidad de las predicciones positivas  
- **Recall**: capacidad de detectar positivos reales  

Un F1 de **0.68** es excelente para un dataset altamente desbalanceado como el de Spotify.

---

### ● ROC-AUC (96.03%)

Mide la capacidad del modelo para distinguir entre las clases HIT y NO HIT.

- AUC = 0.5 → modelo aleatorio  
- AUC = 1.0 → perfecto  

Un AUC de **0.96** indica un modelo **muy robusto** para separar correctamente ambas clases.

---

## 2. Matriz de Confusión

```
[[44096   340]
 [  859  1250]]
```

### Interpretación:

| Caso | Resultado | Significado |
|------|-----------|-------------|
| **True Negative (TN)** | 44,096 | NO HIT clasificados correctamente |
| **False Positive (FP)** | 340 | NO HIT clasificados como HIT |
| **False Negative (FN)** | 859 | HIT perdidos por el modelo |
| **True Positive (TP)** | 1,250 | HIT detectados correctamente |

El modelo:

- Comete **muy pocos falsos positivos** (solo 340).  
- Detecta una buena cantidad de HIT reales (1,250).  
- Aún pierde algunos HIT (859), lo cual es normal en datasets desbalanceados.

---

## 3. Reporte de Clasificación

### Clase **0 — NO HIT**

| Métrica | Valor |
|---------|-------|
| Precision | 0.98 |
| Recall | 0.99 |
| F1-score | 0.99 |

El modelo es casi perfecto clasificando canciones que NO serán hit.

---

### Clase **1 — HIT**

| Métrica | Valor |
|---------|-------|
| Precision | 0.79 |
| Recall | 0.59 |
| F1-score | 0.68 |

Interpretación:

- **Precision 0.79:** Cuando el modelo dice “será un HIT”, acierta el 79% del tiempo.  
- **Recall 0.59:** Detecta el 59% de todos los hits reales.  
- **F1 0.68:** Excelente balance para un dataset tan desbalanceado.

---

## 4. Promedios Globales

| Métrica | Valor |
|---------|-------|
| **Accuracy** | 0.97 |
| **Macro Avg** | 0.83 |
| **Weighted Avg** | 0.97 |

- **Macro avg** refleja el rendimiento equilibrado entre las dos clases.  
- **Weighted avg** está sesgado hacia NO HIT debido al desbalance, pero sigue mostrando un desempeño sobresaliente.

---

## 5. Conclusión Profesional

El modelo LightGBM optimizado:

- Separa muy bien HIT vs NO HIT (AUC = 0.96).  
- Detecta hits con buena precisión y recall para un dataset altamente desbalanceado.  
- Minimiza falsos positivos de forma notable.  
- Mantiene un F1 robusto (0.68), adecuado para la tarea.

Este rendimiento es **excelente** para un modelo de clasificación musical basado en atributos de audio.

---

## 6. Recomendación Final

El modelo está **listo para producción** y para ser integrado en:

- API `/songs/predict_hit`
- Dashboard interactivo “Buscador de Hits”
- Sistemas de recomendación y experimentación musical

Si se desea mejorar aún más el recall de la clase HIT, se pueden ajustar:

- `class_weight`
- técnicas de oversampling (SMOTE)
- thresholds dinámicos por género o época

---
